<a href="https://colab.research.google.com/github/pulkitv/test-rag/blob/main/Audio_Summary_using_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
!pip install openai==0.28
# !pip install pyannote.audio
!pip install openai-whisper

# !pip install torch==1.10.0+cu102
# !pip install pyannote.audio==0.0.1

!pip install git+https://github.com/openai/whisper.git
# !pip install gtts
# !pip install pydub
!pip install faiss-cpu
!pip install huggingface_hub[hf_xet]


  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-pyomhcwo
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-pyomhcwo
  Resolved https://github.com/openai/whisper.git to commit 517a43ecd132a2089d85f4ebc044728a71d49f6e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [23]:
import os
from google.colab import userdata

OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
# GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
# GOOGLE_CSE_ID = userdata.get('GOOGLE_CSE_ID')
# ACTIVELOOP_TOKEN = userdata.get('ACTIVELOOP_TOKEN')
# LANGCHAIN_API_KEY = userdata.get('LANGCHAIN_API_KEY')
# HUGGINGFACE_API_KEY = userdata.get('HUGGINGFACE_API_KEY')
GMAIL_PASSWORD = userdata.get('GMAIL_PASSWORD')


# os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
# os.environ["GOOGLE_CSE_ID"] = GOOGLE_CSE_ID
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
# os.environ["ACTIVELOOP_TOKEN"] = ACTIVELOOP_TOKEN
# os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
# os.environ["HUGGINGFACE_API_KEY"] = HUGGINGFACE_API_KEY
os.environ["GMAIL_PASSWORD"] = GMAIL_PASSWORD

print("Done")

Done


In [24]:
from google.colab import files
import shutil
import torch

# Check if GPU is available (only use GPU-related code in specific cells)
gpu_available = torch.cuda.is_available()

if gpu_available:
    print("GPU is available.")
else:
    print("GPU is not available. Using CPU.")

# Only use GPU if it's available
if gpu_available:
    # Load the Whisper model and move it to GPU
    import whisper
    model = whisper.load_model("base").to(torch.device("cuda"))
    print("Model loaded on GPU.")
    # Upload audio file
    uploaded = files.upload().to(torch.device("cuda"))

else:
    # Load the model on CPU if GPU is not available
    uploaded = files.upload()
    print("Model loaded on CPU.")


# Move the model back to CPU after processing
if gpu_available:
    model.to(torch.device("cpu"))
    print("Model moved back to CPU to free up GPU resources.")
    torch.cuda.empty_cache()
    print("GPU memory cleared.")


# Move the uploaded file to the working directory
audio_file_path = list(uploaded.keys())[0]


GPU is not available. Using CPU.


Saving Weekly review.mp3 to Weekly review (1).mp3
Model loaded on CPU.


In [25]:
import whisper
# from pyannote.audio.pipelines import SpeakerDiarization
# from pyannote.core import Segment
import torch


# Check if GPU is available (only use GPU-related code in specific cells)
gpu_available = torch.cuda.is_available()

if gpu_available:
    print("GPU is available.")
else:
    print("GPU is not available. Using CPU.")

# Only use GPU if it's available
if gpu_available:
    # Load the Whisper model and move it to GPU
    import whisper
    model = whisper.load_model("base").to(torch.device("cuda"))
    print("Model loaded on GPU.")
else:
    # Load the model on CPU if GPU is not available
    model = whisper.load_model("base")
    print("Model loaded on CPU.")

# Perform transcription with Whisper and get word-level timestamps
result = model.transcribe(audio_file_path, word_timestamps=True, task='translate')

# Move the model back to CPU after processing
if gpu_available:
    model.to(torch.device("cpu"))
    print("Model moved back to CPU to free up GPU resources.")

print(result)

# After processing, clear GPU memory
if gpu_available:
    torch.cuda.empty_cache()
    print("GPU memory cleared.")


GPU is not available. Using CPU.
Model loaded on CPU.


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:176: UserWarning: Word-level timestamps on translations may not be reliable.
  warnings.warn("Word-level timestamps on translations may not be reliable.")


{'text': " because it's more scalable. And then web is a kind of like an open journal in the Caldego group, which is only one asset editor. And Caldego is going for IPO. It's linked as a like a latering open. So that's why they want the story of an event. However, web has not been nine engineers. So he first, he doesn't believe that he can build two complete marketplace start in just this nine. Because he also keep improving his existing or rental car app as well. So therefore, that's why he decided to start small by bringing the guy into their location. Then start the marketplace. It's a Zoom car, sure kind of. That's the, that's the thing he versus. Then he found out, 200 people. Like that's a one-beach. That second structure we should be. When he started, he found out that if in the red car yard, blackboard cars and whiteboard cars are part in the same location. Rd you can come to the location. Hey guys, you guys are doing illegal whiteboard. Which is available. If I, if that happen

In [28]:
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer

# Example transcription (replace this with your actual transcription)
transcribed_text = result['text']

# Split the text into smaller chunks (tokens or paragraphs)
def split_text(text, max_chunk_size=4000):
    words = text.split()
    chunks = []
    current_chunk = []
    current_size = 0

    for word in words:
        current_size += len(word) + 1
        if current_size > max_chunk_size:
            chunks.append(" ".join(current_chunk))
            current_chunk = [word]
            current_size = len(word) + 1
        else:
            current_chunk.append(word)

    if current_chunk:
        chunks.append(" ".join(current_chunk))

    return chunks

chunks = split_text(transcribed_text)

# Use a Sentence-BERT model or OpenAI embeddings to convert text to vectors
model = SentenceTransformer('all-MiniLM-L6-v2')
vectors = model.encode(chunks)

# Store vectors in a FAISS index for similarity search
dim = len(vectors[0])  # Vector dimensions
index = faiss.IndexFlatL2(dim)  # L2 distance metric
index.add(np.array(vectors))  # Add vectors to the FAISS index

# Now the vectors are indexed in FAISS
print("Vectors stored in FAISS.")


Vectors stored in FAISS.


In [29]:
# Function to retrieve relevant chunks based on a query
def retrieve_relevant_chunks(query, index, top_k=3):
    query_vector = model.encode([query])  # Convert query to vector
    _, indices = index.search(np.array(query_vector), top_k)  # Search for similar chunks

    relevant_chunks = [chunks[i] for i in indices[0]]
    return relevant_chunks

# Example query to retrieve relevant chunks
query = "You are a personal AI assistant who is hired to take notes and summarize meetings for future reference. Assume that the transcription provided is of a business meeting. You must not generate any content that is not directly supported by the text."
relevant_chunks = retrieve_relevant_chunks(query, index)

# Print relevant chunks
for chunk in relevant_chunks:
    print(chunk)


because it's more scalable. And then web is a kind of like an open journal in the Caldego group, which is only one asset editor. And Caldego is going for IPO. It's linked as a like a latering open. So that's why they want the story of an event. However, web has not been nine engineers. So he first, he doesn't believe that he can build two complete marketplace start in just this nine. Because he also keep improving his existing or rental car app as well. So therefore, that's why he decided to start small by bringing the guy into their location. Then start the marketplace. It's a Zoom car, sure kind of. That's the, that's the thing he versus. Then he found out, 200 people. Like that's a one-beach. That second structure we should be. When he started, he found out that if in the red car yard, blackboard cars and whiteboard cars are part in the same location. Rd you can come to the location. Hey guys, you guys are doing illegal whiteboard. Which is available. If I, if that happens to last v

In [32]:
import openai

# Set OpenAI API key
openai.api_key = OPENAI_API_KEY

# Function to summarize text using GPT-3.5 or GPT-4
def summarize_text(text):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a personal AI assistant who is hired to take notes and summarize meetings for future reference. Assume that the transcription provided is of a business meeting."},
            {"role": "user", "content": f"Summarize in bullet points (within 500 tokens) the following conversation (from an audio recording) without missing any important details. Segregate if there are multiple topics discussed. In each topic, go one level deeper - what was the speaker actually trying to say. At the end of the summary, include any action items that need to be done. Do not include any follow up remarks/questions: {text}"}
        ],
        max_tokens=500
    )
    return response.choices[0].message['content'].strip()

# Summarize the retrieved relevant chunks
final_summary = summarize_text("\n\n".join(relevant_chunks))
print("Final Summary: ", final_summary)


Final Summary:  **Topic 1: Launching a Marketplace**
- The speaker discussed the challenges faced in launching a marketplace due to limited resources and the need to focus on existing projects as well.
- Decision to start small by onboarding a few initial participants before scaling up the marketplace.
- Identified issues with mixed inventory and competition in the marketplace that led to a cautious approach instead of aggressive expansion.

**Topic 2: Marketplace Integration Requirements**
- Requirements for seamless integration included standardized check-in and check-out processes and handling damages.
- Proposal to start a pilot, define integration scope, and commit to a specific number of active cars for testing purposes.
- Consideration for introducing a backup option when cars become unavailable to maintain service continuity.

**Action Items:**
1. Begin the pilot phase and define integration scope with the partner.
2. Confirm commitment to 300 cars for the pilot.
3. Develop a p

In [22]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import os

# Function to send email
def send_email(conversation, summary):
    # Set up the server
    smtp_server = "smtp.gmail.com"  # Using Gmail's SMTP server
    smtp_port = 587  # Standard SMTP port for TLS
    sender_email = "vashishta.pulkit@gmail.com"  # Your Gmail email address
    receiver_email = "vashishta.pulkit@gmail.com"  # Receiver's email address
    email_password = GMAIL_PASSWORD  # Your Gmail App Password (generated from Google account)

    # Create the email content
    subject = "Meeting Summary and Conversation"
    body = f"""
    <h2>Conversation:</h2>
    <pre>{conversation}</pre>

    <h2>Summary:</h2>
    <pre>{summary}</pre>
    """

    # Create MIME message
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = subject

    # Attach the body to the email
    msg.attach(MIMEText(body, "html"))

    # Connect to the server and send the email
    try:
        with smtplib.SMTP(smtp_server, smtp_port) as server:
            server.starttls()  # Secure the connection
            server.login(sender_email, email_password)  # Log in to your email account
            server.sendmail(sender_email, receiver_email, msg.as_string())  # Send the email
            print("Email sent successfully!")
    except Exception as e:
        print(f"Error: {e}")

# Send the email
send_email(chunk, final_summary)


Email sent successfully!
